In [1]:
from unittest.mock import inplace

import pandas as pd
import numpy as np
from features_time_series_generator import *
import pickle as pkl
import ast

In [2]:
file_path_features = 'stock_data_for_emm.pkl'
file_path_stock_1year = 'data/all_stocks_1year_data.csv'

In [3]:
with open(file_path_features, 'rb') as f:
    stock_data_features = pickle.load(f)

stock_data_features = pd.DataFrame(stock_data_features)

In [4]:
stock_data_1year = pd.read_csv(file_path_stock_1year)
stock_data_1year.set_index('stocks', inplace=True)
stock_data_1year

,time_series
stocks,
AOF.DE,"[99.5999984741211, 98.30000305175781, 100.75, ..."
EUK2.HA,"[34.0, 34.0, 30.0, 30.0, 30.0, 30.0, 30.0, 32...."
LFG.MI,"[1.4500000476837158, 1.4800000190734863, 1.450..."
ENV.MI,"[2.880000114440918, 3.0, 2.9800000190734863, 2..."
BAKA.F,"[5.9720001220703125, 5.943999767303467, 5.9699..."
...,...
NBS.L,"[113.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112..."
RICO.L,"[1298.7412109375, 1237.0, 1237.0, 1237.0, 1273..."
KNM.L,"[7887.0, 7887.0, 7887.0, 7887.0, 7847.0, 7847...."


In [5]:
stock_data = stock_data_1year.join(stock_data_features)
stock_data.drop('target', inplace=True, axis=1)
stock_data['target'] = stock_data['time_series'].apply(ast.literal_eval)
stock_data.drop('time_series', inplace=True, axis=1)
stock_data

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,target
stocks,,,,,,,,,,,,
AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,650.0,"[99.5999984741211, 98.30000305175781, 100.75, ..."
EUK2.HA,Germany,Marine Shipping,EUR,Europe/Berlin,HAN,Industrials,0.0,7.614,NaN,41.392,807.0,"[34.0, 34.0, 30.0, 30.0, 30.0, 30.0, 30.0, 32...."
LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,34.0,"[1.4500000476837158, 1.4800000190734863, 1.450..."
ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,232.0,"[2.880000114440918, 3.0, 2.9800000190734863, 2..."
BAKA.F,Spain,Banks—Regional,EUR,Europe/Berlin,FRA,Financial Services,64.0,NaN,5.670958e+09,NaN,6311.0,"[5.9720001220703125, 5.943999767303467, 5.9699..."
...,...,...,...,...,...,...,...,...,...,...,...,...
NBS.L,United Kingdom,Banks—Regional,GBP,Europe/London,LSE,Financial Services,474.0,NaN,1.594289e+09,NaN,17686.0,"[113.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112..."
RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[1298.7412109375, 1237.0, 1237.0, 1237.0, 1273..."
KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[7887.0, 7887.0, 7887.0, 7887.0, 7847.0, 7847...."


In [6]:
def merge_dicts(d1, d2):
    for key, value in d2.items():
        if key in d1:
            # If the value is already a list, append the new value
            if isinstance(d1[key], list):
                d1[key].append(value)
            else:
                # If it's not a list, convert it to a list with the existing and new value
                d1[key] = [d1[key], value]
        else:
            # If the key is not in the dictionary, simply add it
            d1[key] = value
    return d1

In [13]:
new_features = {}
i=0
for time_series in stock_data['target']:
    
    features_timeseries,_,_ = compute_all_features_for_timeseries(time_series, True)
    new_features = merge_dicts(new_features, features_timeseries)
    #print(new_features)
    if i%100==0:
        print(i)
    i+=1
new_features
    

0


C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


100
200


C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


300
400
500
600


...

16300


C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


16400


C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\20203666\Music\envtest\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


{'og_mean': [119.30059,
  29.81255,
  1.63551,
  3.14437,
  6.84235,
  17.35176,
  0.09365,
  0.4876,
  0.00118,
  3.22888,
  162.37647,
  33.80333,
  1.11059,
  7.28804,
  18.0098,
  264.96078,
  80.40016,
  1.08813,
  8.25166,
  5.69247,
  10.104,
  786.39098,
  1.025,
  20.99576,
  82.43259,
  36.54898,
  5.5891,
  84.65549,
  27.60157,
  3.19459,
  106.55847,
  45.00684,
  9.86627,
  29.19596,
  3.94322,
  2.28233,
  103.52674,
  1.60847,
  4.47994,
  6.33851,
  105.34298,
  32.32392,
  750.64902,
  11.76255,
  183.57255,
  31.55608,
  2.52168,
  141.61863,
  6.13141,
  2.98404,
  121.6751,
  3.51584,
  4.49624,
  0.838,
  246.21392,
  4.87173,
  101.68824,
  45.9889,
  61.6451,
  47.04816,
  69.29016,
  16.17041,
  54.8018,
  0.15553,
  33.83216,
  194.05098,
  0.06629,
  24.93459,
  0.07847,
  55.47333,
  89.08039,
  58.15276,
  2.44519,
  66.12031,
  224.95745,
  32.28455,
  16.74065,
  0.01863,
  0.14273,
  97.06471,
  0.51009,
  102.63302,
  42.56667,
  145.06,
  71.45373,
  0

In [20]:
stock_data_all_features = stock_data.assign(**new_features)
stock_data_all_features

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,...,growth_std_dev,growth_autocorr_lag1,growth_min,growth_max,growth_range,growth_window_min,growth_window_max,growth_trend_slope,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc
stocks,,,,,,,,,,,,,,,,,,,,,
AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,...,1.89458,0.97943,-11.01056,6.73235,17.74291,237,205,-0.00104,16.82441,-17.72197
EUK2.HA,Germany,Marine Shipping,EUR,Europe/Berlin,HAN,Industrials,0.0,7.614,NaN,41.392,...,2.21488,0.90756,-11.76471,8.66666,20.43137,3,248,0.00143,8.66666,-11.76471
LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,...,2.18054,0.95050,-8.00000,12.17949,20.17949,56,188,-0.00086,18.59324,-8.00000
ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,...,1.57873,0.90582,-6.57895,5.55555,12.13450,17,16,-0.00075,11.23512,-6.57895
BAKA.F,Spain,Banks—Regional,EUR,Europe/Berlin,FRA,Financial Services,64.0,NaN,5.670958e+09,NaN,...,1.55239,0.99193,-5.23102,4.80802,10.03904,213,13,-0.00024,11.12352,-12.02754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NBS.L,United Kingdom,Banks—Regional,GBP,Europe/London,LSE,Financial Services,474.0,NaN,1.594289e+09,NaN,...,0.41586,0.99734,-1.85185,2.82258,4.67443,110,52,-0.00086,5.28340,-1.85185
RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,...,1.80197,0.97884,-9.63956,12.64970,22.28926,52,235,0.00194,13.67960,-9.63956
KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,...,1.76817,0.99556,-8.39559,9.76675,18.16234,39,159,0.00133,9.76675,-8.39559


In [21]:
#stock_data_all_features.to_pickle('stock_data_1year.pkl')


In [22]:
file_path = 'stock_data_1year.pkl'
with open(file_path, 'rb') as f:
    stock_data_new = pickle.load(f)

stock_data_new = pd.DataFrame(stock_data_new)
stock_data_new

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,...,growth_std_dev,growth_autocorr_lag1,growth_min,growth_max,growth_range,growth_window_min,growth_window_max,growth_trend_slope,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc
stocks,,,,,,,,,,,,,,,,,,,,,
AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,...,1.89458,0.97943,-11.01056,6.73235,17.74291,237,205,-0.00104,16.82441,-17.72197
EUK2.HA,Germany,Marine Shipping,EUR,Europe/Berlin,HAN,Industrials,0.0,7.614,NaN,41.392,...,2.21488,0.90756,-11.76471,8.66666,20.43137,3,248,0.00143,8.66666,-11.76471
LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,...,2.18054,0.95050,-8.00000,12.17949,20.17949,56,188,-0.00086,18.59324,-8.00000
ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,...,1.57873,0.90582,-6.57895,5.55555,12.13450,17,16,-0.00075,11.23512,-6.57895
BAKA.F,Spain,Banks—Regional,EUR,Europe/Berlin,FRA,Financial Services,64.0,NaN,5.670958e+09,NaN,...,1.55239,0.99193,-5.23102,4.80802,10.03904,213,13,-0.00024,11.12352,-12.02754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NBS.L,United Kingdom,Banks—Regional,GBP,Europe/London,LSE,Financial Services,474.0,NaN,1.594289e+09,NaN,...,0.41586,0.99734,-1.85185,2.82258,4.67443,110,52,-0.00086,5.28340,-1.85185
RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,...,1.80197,0.97884,-9.63956,12.64970,22.28926,52,235,0.00194,13.67960,-9.63956
KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,...,1.76817,0.99556,-8.39559,9.76675,18.16234,39,159,0.00133,9.76675,-8.39559
